#  Evaluation Demo

### Imports

In [1]:
import io
import base64
import glob
import os

import gzip
import json
import numpy as np
import torch
from IPython.display import HTML
from skvideo.io import FFmpegReader, ffprobe, vwrite
from torch.autograd import Variable
from ptcap.trainers import DataParallelWrapper
from ptcap.scores import ( caption_accuracy, first_token_accuracy, token_accuracy)
from ptcap.data.annotation_parser import JsonParser
from collections import OrderedDict
from collections import Counter, namedtuple



/home/farzaneh/venv/ptcap/lib/python3.5/site-packages/skvideo/__init__.py:356: UserWarning: avconv/avprobe not found in path: 
  warnings.warn("avconv/avprobe not found in path: " + str(path), UserWarning)


In [53]:
int2label = {
     0: 'Approaching [something] with your camera',
     1: 'Attaching [something] to [something]',
     2: 'Bending [something] so that it deforms',
     3: 'Bending [something] until it breaks',
     4: 'Burying [something] in [something]',
     5: 'Closing [something]',
     6: 'Covering [something] with [something]',
     7: 'Digging [something] out of [something]',
     8: 'Dropping [something] behind [something]',
     9: 'Dropping [something] in front of [something]',
     10: 'Dropping [something] into [something]',
     11: 'Dropping [something] next to [something]',
     12: 'Dropping [something] onto [something]',
     13: 'Failing to put [something] into [something] because [something] does not fit',
     14: 'Folding [something]',
     15: 'Hitting [something] with [something]',
     16: 'Holding [something]',
     17: 'Holding [something] behind [something]',
     18: 'Holding [something] in front of [something]',
     19: 'Holding [something] next to [something]',
     20: 'Holding [something] over [something]',
     21: 'Laying [something] on the table on its side, not upright',
     22: 'Letting [something] roll along a flat surface',
     23: 'Letting [something] roll down a slanted surface',
     24: 'Letting [something] roll up a slanted surface, so it rolls back down',
     25: 'Lifting [something] up completely without letting it drop down',
     26: 'Lifting [something] up completely, then letting it drop down',
     27: 'Lifting [something] with [something] on it',
     28: 'Lifting a surface with [something] on it but not enough for it to slide down',
     29: 'Lifting a surface with [something] on it until it starts sliding down',
     30: 'Lifting up one end of [something] without letting it drop down',
     31: 'Lifting up one end of [something], then letting it drop down',
     32: 'Moving [part] of [something]',
     33: 'Moving [something] across a surface until it falls down',
     34: 'Moving [something] across a surface without it falling down',
     35: 'Moving [something] and [something] away from each other',
     36: 'Moving [something] and [something] closer to each other',
     37: 'Moving [something] and [something] so they collide with each other',
     38: 'Moving [something] and [something] so they pass each other',
     39: 'Moving [something] away from [something]',
     40: 'Moving [something] away from the camera',
     41: 'Moving [something] closer to [something]',
     42: 'Moving [something] down',
     43: 'Moving [something] towards the camera',
     44: 'Moving [something] up',
     45: 'Moving away from [something] with your camera',
     46: 'Opening [something]',
     47: 'Picking [something] up',
     48: 'Piling [something] up',
     49: 'Plugging [something] into [something]',
     50: 'Plugging [something] into [something] but pulling it right out as you remove your hand',
     51: 'Poking [something] so it slightly moves',
     52: 'Poking [something] so lightly that it doesn’t or almost doesn’t move',
     53: 'Poking [something] so that it falls over',
     54: 'Poking [something] so that it spins around',
     55: 'Poking a hole into [some substance]',
     56: 'Poking a hole into [something soft]',
     57: 'Poking a stack of [something] so the stack collapses',
     58: 'Poking a stack of [something] without the stack collapsing',
     59: 'Pouring [something] into [something]',
     60: 'Pouring [something] into [something] until it overflows',
     61: 'Pouring [something] onto [something]',
     62: 'Pouring [something] out of [something]',
     63: 'Pretending or failing to wipe [something] off of [something]',
     64: 'Pretending or trying and failing to twist [something]',
     65: 'Pretending to be tearing [something that is not tearable]',
     66: 'Pretending to close [something] without actually closing it',
     67: 'Pretending to open [something] without actually opening it',
     68: 'Pretending to pick [something] up',
     69: 'Pretending to poke [something]',
     70: 'Pretending to pour [something] out of [something], but [something] is empty',
     71: 'Pretending to put [something] behind [something]',
     72: 'Pretending to put [something] into [something]',
     73: 'Pretending to put [something] next to [something]',
     74: 'Pretending to put [something] on a surface',
     75: 'Pretending to put [something] onto [something]',
     76: 'Pretending to put [something] underneath [something]',
     77: 'Pretending to scoop [something] up with [something]',
     78: 'Pretending to spread “air” onto [something]',
     79: 'Pretending to sprinkle "air" onto [something]',
     80: 'Pretending to squeeze [something]',
     81: 'Pretending to take [something] from [somewhere]',
     82: 'Pretending to take [something] out of [something]',
     83: 'Pretending to throw [something]',
     84: 'Pretending to turn [something] upside down',
     85: 'Pulling [something] from behind of [something]',
     86: 'Pulling [something] from left to right',
     87: 'Pulling [something] from right to left',
     88: 'Pulling [something] onto [something]',
     89: 'Pulling [something] out of [something]',
     90: 'Pulling two ends of [something] but nothing happens',
     91: 'Pulling two ends of [something] so that it gets stretched',
     92: 'Pulling two ends of [something] so that it separates into two pieces',
     93: 'Pushing [something] from left to right',
     94: 'Pushing [something] from right to left',
     95: 'Pushing [something] off of [something]',
     96: 'Pushing [something] onto [something]',
     97: 'Pushing [something] so it spins',
     98: "Pushing [something] so that it almost falls off but doesn't",
     99: 'Pushing [something] so that it falls off the table',
     100: 'Pushing [something] so that it slightly moves',
     101: 'Pushing [something] with [something]',
     102: 'Putting [number of] [something] onto [something]',
     103: 'Putting [something similar to other things that are already on the table]',
     104: 'Putting [something that cannot actually stand upright] upright on the table, so it falls on its side',
     105: 'Putting [something] and [something] on the table',
     106: 'Putting [something] behind [something]',
     107: 'Putting [something] in front of [something]',
     108: 'Putting [something] into [something]',
     109: 'Putting [something] next to [something]',
     110: 'Putting [something] on a flat surface without letting it roll',
     111: 'Putting [something] on a surface',
     112: 'Putting [something] on the edge of [something] so it is not supported and falls down',
     113: 'Putting [something] onto [something else that cannot support it] so it falls down',
     114: 'Putting [something] onto [something]',
     115: 'Putting [something] onto a slanted surface but it doesn’t glide down',
     116: 'Putting [something] that can’t roll onto a slanted surface, so it slides down',
     117: 'Putting [something] that can’t roll onto a slanted surface, so it stays where it is',
     118: 'Putting [something] underneath [something]',
     119: 'Putting [something] upright on the table',
     120: 'Putting [something], [something] and [something] on the table',
     121: 'Removing [something], revealing [something] behind',
     122: 'Rolling [something] on a flat surface',
     123: 'Scooping [something] up with [something]',
     124: 'Show a shadow of [something] that is moving. ',
     125: 'Show a shadow of [something], making sure the shadow is not moving. ',
     126: 'Showing [something] behind [something]',
     127: 'Showing [something] next to [something]',
     128: 'Showing [something] on top of [something]',
     129: 'Showing [something] to the camera',
     130: 'Showing a photo of [something] to the camera',
     131: 'Showing a shadow of [something] that is moving.',
     132: 'Showing a shadow of [something], making sure the shadow is not moving.',
     133: 'Showing that [something] is empty',
     134: 'Showing that [something] is inside [something]',
     135: 'Spilling [something] behind [something]',
     136: 'Spilling [something] next to [something]',
     137: 'Spilling [something] onto [something]',
     138: 'Spinning [something] so it continues spinning',
     139: 'Spinning [something] that quickly stops spinning',
     140: 'Spreading [something] onto [something]',
     141: 'Sprinkling [something] onto [something]',
     142: 'Squeezing [something]',
     143: 'Stacking [number of] [something]',
     144: 'Stuffing [something] into [something]',
     145: 'Taking [one of many similar things on the table]',
     146: 'Taking [something] from [somewhere]',
     147: 'Taking [something] out of [something]',
     148: 'Tearing [something] into two pieces',
     149: 'Tearing [something] just a little bit',
     150: 'Throwing [something]',
     151: 'Throwing [something] against [something]',
     152: 'Throwing [something] in the air and catching it',
     153: 'Throwing [something] in the air and letting it fall',
     154: 'Throwing [something] onto a surface',
     155: 'Tilting [something] with [something] on it slightly so it doesn’t fall down',
     156: 'Tilting [something] with [something] on it until it falls off',
     157: 'Tipping [something] over',
     158: 'Tipping [something] with [something in it] over, so [something in it] falls out',
     159: 'Touching (without moving) [part] of [something]',
     160: 'Trying but failing to attach [something] to [something] because it doesn’t stick',
     161: 'Trying to bend [something unbendable] so nothing happens',
     162: 'Trying to pour [something] into [something], but missing so it spills next to it',
     163: 'Turning [something] upside down',
     164: 'Turning the camera downwards while filming [something]',
     165: 'Turning the camera left while filming [something]',
     166: 'Turning the camera right while filming [something]',
     167: 'Turning the camera upwards while filming [something]',
     168: 'Twisting (wringing) [something] wet until water comes out',
     169: 'Twisting [something]',
     170: 'Uncovering [something]',
     171: 'Unfolding [something]',
     172: 'Wiping [something] off of [something]',
     173: '[Something] being deflected from [something]',
     174: '[Something] colliding with [something] and both are being deflected',
     175: '[Something] colliding with [something] and both come to a halt',
     176: '[Something] falling like a feather or paper',
     177: '[Something] falling like a rock'
}

In [54]:
stop_words = ['A', 'AN', 'THE', '<END>']

def safe_div(x,y):
    if y == 0:
        return 0
    return x / y
def fscore(precision, recall, beta=1):
    numerator = (1.0 + (beta ** 2)) * precision * recall
    denominator = ((beta ** 2) * precision) + recall
    return {"fscore": safe_div(numerator, denominator)}
class LCS(object):
    """
    The main functionality of this class is to compute the LCS (Lowest Common
    Subsequence) between a caption and prediction. By default, it returns the
    precision and recall values calculated based on the LCS between a prediction
    and a caption.
    """
    def __init__(self, functions_list, tokenizer):
        """
        Initializes functions_list and tokenizer.
        Args:
        functions_list: A list of the functions that will be applied on the
        precision and recall values calculated based on the LCS between a
        prediction and a caption.
        """

        self.functions_list = functions_list
        self.scores_container = OrderedDict()
        self.scores_dict = OrderedDict()
        self.tokenizer = tokenizer

    def __call__(self, outputs):
        string_predictions = [self.tokenizer.get_string(str_pred.data.numpy())
                              for str_pred in outputs.predictions]
        return self.score_batch(string_predictions, outputs.string_captions)

    def collect_scores(self, batch_scores_dict, scores_dict):
        for metric, metric_value in scores_dict.items():
            if metric not in batch_scores_dict:
                batch_scores_dict[metric] = [metric_value]
            else:
                batch_scores_dict[metric].append(metric_value)
        return batch_scores_dict

    @classmethod
    def compute_lcs(cls, prediction, caption):
        num_rows = len(prediction)
        num_cols = len(caption)

        table = [[0] * (num_cols + 1) for _ in range(num_rows + 1)]
        for i in range(1, num_rows + 1):
            for j in range(1, num_cols + 1):
                if prediction[i - 1] == caption[j - 1]:
                    table[i][j] = table[i - 1][j - 1] + 1
                else:
                    table[i][j] = max(table[i][j - 1], table[i - 1][j])
        return table, table[num_rows][num_cols]

    def mean_scores(self, batch_scores_dict):
        for metric, metric_value in batch_scores_dict.items():
            batch_scores_dict[metric] = np.mean(metric_value)
        return batch_scores_dict

    def score_batch(self, predictions, captions):
        assert len(predictions) == len(captions)

        batch_scores_dict = OrderedDict()
        for count, (prediction, caption) in enumerate(zip(predictions,
                                                          captions)):
            scores_dict = self.score_sample(prediction.split(), caption.split())
            batch_scores_dict = self.collect_scores(batch_scores_dict,
                                                    scores_dict)

        batch_scores_dict = self.mean_scores(batch_scores_dict)
        return batch_scores_dict

    def score_sample(self, prediction, caption):
        scores_dict = OrderedDict()
        _, lcs_score = self.compute_lcs(prediction, caption)
        scores_dict["precision"] = safe_div(lcs_score, len(prediction))
        scores_dict["recall"] = safe_div(lcs_score, len(caption))

        for score_function in self.functions_list:
            scores_dict.update(score_function(scores_dict["precision"],
                                              scores_dict["recall"]))

        return scores_dict

### Tool to deal with mpeg videos

In [55]:
def show_video(video_filenames):
    """
    Tool to display videos inside the notebook.
    """
    
    if type(video_filenames) is not list:
        video_filenames = [video_filenames]
    
    html_code = ''
    for filename in video_filenames:
        video = io.open(filename, 'r+b').read()
        encoded = base64.b64encode(video)
        html_code += '''
        <video alt="test" width="640" height="480" controls>
        <source src="data:video/mp4;base64,{0}" type="video/mp4" />
        </video>
        '''.format(encoded.decode('ascii'))
        
    return HTML(data= html_code)


def open_mpeg_video(fname, framerate, size):
    """
    Open an mpeg video, and return it as a numpy array.
    """
    
    metadata = ffprobe(fname)
    duration = float(metadata['video']['@duration'])
    # Compute corresponding nb of frames
    nframes = int(duration * framerate)
    oargs = {
        "-r": "%d" % framerate,
        "-vframes": "%d" % nframes,
        "-s": "%dx%d" % (size[0], size[1])
    }
    # Open file
    reader = FFmpegReader(fname, inputdict={}, outputdict=oargs)
    video = []
    # Get frames until there is no more
    for frame in reader.nextFrame():
        video.append(frame)
    # Return as a numpy array
    return np.array(video)

## A) Model Instantiation

In [56]:
from ptcap.model.captioners import EncoderDecoder
from ptcap.model.encoders import CNN3dLSTMEncoder
from ptcap.model.pretrained_encoders import FCEncoder, JesterEncoder, BIJesterEncoder
from ptcap.model.decoders import LSTMDecoder, CoupledLSTMDecoder

#net = FullyConvolutionalNet(num_classes=178)jester1024_cutoff_300_ssssssss/

net = EncoderDecoder(
        encoder=BIJesterEncoder,
        decoder=CoupledLSTMDecoder,
        encoder_kwargs={"freeze": False},#, "pretrained_path": "/home/farzaneh/PycharmProjects/pretrained_nets/fully_conv_net_on_smtsmt_20170627/model.checkpoint"},
        decoder_kwargs={"embedding_size": 256, "hidden_size": 1024, "num_lstm_layers": 2, 
        "vocab_size": 2986, "num_step" :17}, 
        gpus=[0]).cuda()
net = DataParallelWrapper(net, device_ids=[0]).cuda(0)

gpus: [0]


## B) Loading weights

In [6]:
path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/clapnet_classif0.01_caption1_step2/'
# path = '/home/farzaneh/PycharmProjects/pytorch-captioning/results/clapnet_captioning_only_f0.1'

checkpoint = torch.load(path + '/model.best')


net.load_state_dict(checkpoint["model"])


## C) Load Tokenizer

In [7]:
from ptcap.data.tokenizer import Tokenizer

USER_MAXLEN=17
tokenizer = Tokenizer(user_maxlen=USER_MAXLEN)
tokenizer.load_dictionaries(path)

## D) Testing 

In [8]:
TMP_VIDEO_FILENAME = 'tmp.mp4'

def unpreprocess(video):
    video = video.data.numpy()[0]
    video = 64. * video.transpose(1, 2, 3, 0)
    return np.array(video, 'uint8')

def demo(net, preprocessor, filename, top_n=5):
    # Open mpeg file and get a numpy array
    video_uint8 = open_mpeg_video(filename, 12, [128, 128])
    # Preprocessing
    video = preprocessor(video_uint8)
    # Convert to torch variable
    video = Variable(torch.from_numpy(video[None]), volatile=True).cuda()
    empty_caption = Variable(torch.zeros([1, 1]), volatile=True).long().cuda()
    
    # Compute predictions
    pred, class_pred = net.forward((video, empty_caption), use_teacher_forcing=False)
    # Convert to numpy 
    pred = np.exp(pred.cpu().data.numpy())[0]
        
    pred_argmax = np.argmax(pred, axis=1)
    decoded_pred = tokenizer.decode_caption(pred_argmax)
    beautiful_caption = " ".join(str(e+" ") for e in decoded_pred if "<END>" not in e)
    #print('__CAPTION__: {}'.format(beautiful_caption))
    
    
    # Class index
    class_index = torch.max(class_pred, dim=1)[1].cpu().data[0]
    cls = int2label[class_index]
    #print('ACTION: {:60s}\n'.format(cls))
    
    
    matched_action = get_template(decoded_pred, templates, tokenizer)
    # print(actions)
    objects = get_object_tokens(decoded_pred, matched_action[0][0])

    
    objects_list = extract_objects(objects)
    # Print class name with proba
    # Save input video in tmp file
    vwrite(TMP_VIDEO_FILENAME, unpreprocess(video.cpu()))
    return beautiful_caption, cls, objects_list, matched_action[0][0]


def path_generator(annotation_path, root_path):
    with gzip.open(annotation_path, "rt") as f:
        annotations = json.load(f)
    files = [elem['file'] for elem in annotations]
    labels = [elem['label'] for elem in annotations]
    placeholders = [elem['placeholders'] for elem in annotations] 
    actions = [elem['template'].replace("[","").replace("]", "").replace(",","").upper() for elem in annotations]
    
    return ((os.path.join(root_path, f), label, a, p) for f,label,p, a in zip(files, labels, placeholders, actions))

In [9]:
# Path generator
path_gen = path_generator('/data/20bn-somethingsomething/json/test_20170929.json.gz', 
                          '/data/20bn-somethingsomething/videos')
# Put the netwoark in evaluation mode
_ = net.eval()

#### Preprocessor

In [10]:
from rtorchn.data.preprocessing import default_evaluation_preprocesser

preprocessor = default_evaluation_preprocesser([48, 96, 96], 64.)

In [11]:
for i in range(5000):
    path_to_video, label, _,_ = next(path_gen)

## Longest Common Subsequence

In [12]:
def get_template(candidates, templates, tokenizer):
   
    lcs = LCS([fscore], tokenizer)
    
    max_templates = []
    #print("There are {} templates".format(len(templates)))

    candidates = [" ".join(candidates)]
    for candidate in candidates:
        
        max_lcs_template = ""
        max_lcs_value = -1
        for template in templates:
            lcs_value = compute_LCS(lcs, candidate, template, tokenizer)
            if lcs_value > max_lcs_value:
                max_lcs_template = template
                max_lcs_value = lcs_value
        max_templates.append((max_lcs_template, max_lcs_value))
#         print("Candidate: {}".format(candidate))
        #print("MATCHED ACTION : {}".format(max_lcs_template))

    return max_templates


def compute_LCS(lcs, candidate, template, tokenizer):
    encoded_caption = Variable(
        torch.LongTensor([tokenizer.encode_caption(candidate)]))
    encoded_prediction = Variable(
        torch.LongTensor([tokenizer.encode_caption(template)]))
    score_attr = namedtuple("ScoresAttr", "string_captions captions predictions")
    in_tuple = score_attr([candidate], encoded_caption, encoded_prediction)
    lcs_output = lcs(in_tuple)
    return lcs_output['fscore']

def extract_objects(object_tokens_list):
    
    objects_list = []
    if len(object_tokens_list) == 0:
        return objects_list
    
    next_token_ind =  object_tokens_list[0][0]
    current_object = ""
    for  (ind, token) in object_tokens_list:
        if  next_token_ind == ind:
            current_object += token+" "
        else:
            objects_list.append(current_object+" ")
            current_object = token
            next_token_ind = ind
        next_token_ind += 1
        
        
    if len(current_object)>0:
        objects_list.append(current_object)
               
    #print("PREDICTED OBJECTS: {}".format(objects_list))
    return objects_list



def get_object_tokens(caption, template):
    return [(i,token) for (i,token) in enumerate(caption) if token not in template and token not in stop_words]

In [13]:
articles= ["the", "a", "an", "A", "An", "The"]

annotations = JsonParser.open_annotation("/data/20bn-somethingsomething/json/train_20171031.json.gz")
templates = np.unique(annotations["template"]) # A list of templates
objects = annotations["placeholders"]
obj_tokens = [token for token in objects if token not in stop_words]
all_obj=[item for sublist in objects for item in sublist]
filtered_obj =  [" ".join(obj) for obj in all_obj for  token in obj if token not in articles]
templates = [" ".join(tokenizer.tokenize(t)) for t in templates]


# get_objects(sentences1[0].split(), sentence1_templates[0].split())



In [14]:
articles= ["the", "a", "an", "A", "An", "The"]

fil3 = [token.upper() for obj in all_obj for token in obj.split(" ")]
fil2 = [[token for token in obj.split(" ") if token not in articles ] for obj in all_obj]
fil = list( map(lambda p:" ".join([token for token in p.split(" ") if token not in articles ]), all_obj))



In [29]:
all_objects = {}
correct_objects = {}
all_actions = {}
correct_actions = {}

for i in range(1000):
    path_to_video, target_caption, target_action, target_objects = next(path_gen)
    for p in target_objects:
        p_tokens = p.split(" ")
        for pto in p_tokens:
            all_objects[pto.strip().upper()] = all_objects.get(pto.strip().upper(), 0) + 1
        
    all_actions[target_action] = all_actions.get(target_action, 0) + 1

   

    pred_caption, pred_action, pred_objects, matched_action = demo(net, preprocessor, path_to_video)
    for (i,o) in enumerate(pred_objects):
        o_tokens = o.strip().split(" ")
        for oto in o_tokens:
            if i<len(target_objects) and oto in target_objects[i].upper():
                correct_objects[oto] = correct_objects.get(oto, 0) + 1
                print("woohoo")
               
            
    print('TARGET CAPTION: {}'.format(target_caption))
    print('PRED   CAPTION: {}\n'.format(pred_caption))
    print('TARGET  ACTION: {}'.format(target_action))
    print('CLASSIF ACTION: {}'.format(pred_action))
    print('CAPTION ACTION: {}\n'.format(matched_action))
            
    print('TARGET OBJECTS: {}'.format(target_objects))
    print('PRED   OBJECTS: {}\n'.format(pred_objects))


    if matched_action == target_action:
        print("yesss")
        correct_actions[matched_action] = correct_actions.get(matched_action, 0) + 1
        
    print('{}\n'.format('-'*65))
    
print(all_objects)
print(correct_objects)

print(all_actions)
print(correct_actions)
    

woohoo
TARGET CAPTION: tearing tissue paper just a little bit
PRED   CAPTION: TEARING  A  PAPER  JUST  A  LITTLE  BIT 

TARGET  ACTION: TEARING SOMETHING JUST A LITTLE BIT
CLASSIF ACTION: Tearing [something] just a little bit
CAPTION ACTION: TEARING SOMETHING JUST A LITTLE BIT

TARGET OBJECTS: ['tissue paper']
PRED   OBJECTS: ['PAPER ']

yesss
-----------------------------------------------------------------

TARGET CAPTION: taking scissor out of box
PRED   CAPTION: TAKING  A  SPOON  FROM  A  BOWL 

TARGET  ACTION: TAKING SOMETHING OUT OF SOMETHING
CLASSIF ACTION: Taking [something] from [somewhere]
CAPTION ACTION: TAKING SOMETHING FROM SOMEWHERE

TARGET OBJECTS: ['scissor', 'box']
PRED   OBJECTS: ['SPOON  ', 'BOWL']

-----------------------------------------------------------------

woohoo
TARGET CAPTION: lifting book with bat on it
PRED   CAPTION: PULLING  A  BOOK  OUT  OF  THE  SHELF 

TARGET  ACTION: LIFTING SOMETHING WITH SOMETHING ON IT
CLASSIF ACTION: Pulling [something] out of 

woohoo
TARGET CAPTION: pouring water onto wood
PRED   CAPTION: SPILLING  WATER  NEXT  TO  A  COIN 

TARGET  ACTION: POURING SOMETHING ONTO SOMETHING
CLASSIF ACTION: Spilling [something] next to [something]
CAPTION ACTION: SPILLING SOMETHING NEXT TO SOMETHING

TARGET OBJECTS: ['water', 'wood']
PRED   OBJECTS: ['WATER  ', 'COIN']

-----------------------------------------------------------------

TARGET CAPTION: moving scissors up
PRED   CAPTION: MOVING  A  MOBILE  PHONE  UP 

TARGET  ACTION: MOVING SOMETHING UP
CLASSIF ACTION: Moving [something] up
CAPTION ACTION: MOVING SOMETHING UP

TARGET OBJECTS: ['scissors']
PRED   OBJECTS: ['MOBILE PHONE ']

yesss
-----------------------------------------------------------------

TARGET CAPTION: putting toy on a surface
PRED   CAPTION: PUTTING  A  COIN  UPRIGHT  ON  THE  TABLE 

TARGET  ACTION: PUTTING SOMETHING ON A SURFACE
CLASSIF ACTION: Putting [something] on a surface
CAPTION ACTION: PUTTING SOMETHING UPRIGHT ON THE TABLE

TARGET OBJECTS: ['t

TARGET CAPTION: pretending to take Chocolate  from Box
PRED   CAPTION: PUTTING  A  <UNK> 

TARGET  ACTION: PRETENDING TO TAKE SOMETHING FROM SOMEWHERE
CLASSIF ACTION: Attaching [something] to [something]
CAPTION ACTION: PUTTING SOMETHING ON A SURFACE

TARGET OBJECTS: ['Chocolate ', 'Box']
PRED   OBJECTS: ['<UNK> ']

-----------------------------------------------------------------

TARGET CAPTION: holding Remote behind Mobile
PRED   CAPTION: PRETENDING  TO  PUT  A  PLASTIC  BOTTLE  BEHIND  A  CARDBOARD  BOX 

TARGET  ACTION: HOLDING SOMETHING BEHIND SOMETHING
CLASSIF ACTION: Holding [something] behind [something]
CAPTION ACTION: PRETENDING TO PUT SOMETHING BEHIND SOMETHING

TARGET OBJECTS: ['Remote', 'Mobile']
PRED   OBJECTS: ['PLASTIC BOTTLE  ', 'CARDBOARDBOX ']

-----------------------------------------------------------------

TARGET CAPTION: squeezing a stuffed animal
PRED   CAPTION: SQUEEZING  A  BOTTLE 

TARGET  ACTION: SQUEEZING SOMETHING
CLASSIF ACTION: Squeezing [something]
CA

TARGET CAPTION: holding A stick
PRED   CAPTION: HOLDING  A  SET  OF  COLOURING  MARKER  NEXT  TO  A  SET  OF  PENCIL 

TARGET  ACTION: HOLDING SOMETHING
CLASSIF ACTION: Putting [something that cannot actually stand upright] upright on the table, so it falls on its side
CAPTION ACTION: HOLDING SOMETHING NEXT TO SOMETHING

TARGET OBJECTS: ['A stick']
PRED   OBJECTS: ['SET OF COLOURING MARKER  ', 'SETOF PENCIL ']

-----------------------------------------------------------------

TARGET CAPTION: showing a glas bottle behind a wire
PRED   CAPTION: SHOWING  A  SHADOW  OF  CHAIR  MAKING  SURE  THE  SHADOW  IS  NOT  MOVING 

TARGET  ACTION: SHOWING SOMETHING BEHIND SOMETHING
CLASSIF ACTION: Showing [something] behind [something]
CAPTION ACTION: SHOWING A SHADOW OF SOMETHING MAKING SURE THE SHADOW IS NOT MOVING

TARGET OBJECTS: ['a glas bottle', 'a wire']
PRED   OBJECTS: ['CHAIR ']

-----------------------------------------------------------------

TARGET CAPTION: hitting Chair with Notebook 


TARGET CAPTION: tearing paper towel  into two pieces
PRED   CAPTION: MOVING  A  BALL  AND  A  BALL  AWAY  FROM  EACH  OTHER 

TARGET  ACTION: TEARING SOMETHING INTO TWO PIECES
CLASSIF ACTION: Moving [something] and [something] away from each other
CAPTION ACTION: MOVING SOMETHING AND SOMETHING AWAY FROM EACH OTHER

TARGET OBJECTS: ['paper towel ']
PRED   OBJECTS: ['BALL  ', 'BALL']

-----------------------------------------------------------------

TARGET CAPTION: squeezing purse
PRED   CAPTION: SQUEEZING  A  PLASTIC  CUP 

TARGET  ACTION: SQUEEZING SOMETHING
CLASSIF ACTION: Squeezing [something]
CAPTION ACTION: SQUEEZING SOMETHING

TARGET OBJECTS: ['purse']
PRED   OBJECTS: ['PLASTIC CUP ']

yesss
-----------------------------------------------------------------

TARGET CAPTION: lifting wallet with hair clip on it
PRED   CAPTION: MOVING  A  BOOK  UP 

TARGET  ACTION: LIFTING SOMETHING WITH SOMETHING ON IT
CLASSIF ACTION: Moving [something] up
CAPTION ACTION: MOVING SOMETHING UP

TARGET

...

woohoo
TARGET CAPTION: pouring Water into Toilet
PRED   CAPTION: POURING  WATER  INTO  A  CUP 

TARGET  ACTION: POURING SOMETHING INTO SOMETHING
CLASSIF ACTION: Pouring [something] into [something]
CAPTION ACTION: POURING SOMETHING INTO SOMETHING

TARGET OBJECTS: ['Water', 'Toilet']
PRED   OBJECTS: ['WATER  ', 'CUP']

yesss
-----------------------------------------------------------------

TARGET CAPTION: plugging hair straightener into wall outlet
PRED   CAPTION: TWISTING  KNOB 

TARGET  ACTION: PLUGGING SOMETHING INTO SOMETHING
CLASSIF ACTION: Plugging [something] into [something]
CAPTION ACTION: TWISTING SOMETHING

TARGET OBJECTS: ['hair straightener', 'wall outlet']
PRED   OBJECTS: ['KNOB ']

-----------------------------------------------------------------

TARGET CAPTION: twisting twisting a cap to install
PRED   CAPTION: TEARING  A  PAPER  INTO  TWO  PIECES 

TARGET  ACTION: TWISTING SOMETHING
CLASSIF ACTION: Tearing [something] into two pieces
CAPTION ACTION: TEARING SOMETHING 

TARGET CAPTION: letting a ball roll along a flat surface
PRED   CAPTION: LETTING  A  TOY  CAR  ROLL  ALONG  A  FLAT  SURFACE 

TARGET  ACTION: LETTING SOMETHING ROLL ALONG A FLAT SURFACE
CLASSIF ACTION: Letting [something] roll along a flat surface
CAPTION ACTION: LETTING SOMETHING ROLL ALONG A FLAT SURFACE

TARGET OBJECTS: ['a ball']
PRED   OBJECTS: ['TOY CAR ']

yesss
-----------------------------------------------------------------

TARGET CAPTION: showing Pillow behind Cushion
PRED   CAPTION: SHOWING  A  SHADOW  OF  A  HAND  MAKING  SURE  THE  SHADOW  IS  NOT 

TARGET  ACTION: SHOWING SOMETHING BEHIND SOMETHING
CLASSIF ACTION: Showing a shadow of [something], making sure the shadow is not moving.
CAPTION ACTION: SHOWING A SHADOW OF SOMETHING MAKING SURE THE SHADOW IS NOT MOVING

TARGET OBJECTS: ['Pillow', 'Cushion']
PRED   OBJECTS: ['HAND ']

-----------------------------------------------------------------

TARGET CAPTION: moving Comb up
PRED   CAPTION: MOVING  A  BOOK  UP 

TARGE

woohoo
TARGET CAPTION: pouring water out of cup
PRED   CAPTION: POURING  WATER  OUT  OF  PLASTIC  CUP 

TARGET  ACTION: POURING SOMETHING OUT OF SOMETHING
CLASSIF ACTION: Pouring [something] out of [something]
CAPTION ACTION: POURING SOMETHING OUT OF SOMETHING

TARGET OBJECTS: ['water', 'cup']
PRED   OBJECTS: ['WATER  ', 'PLASTICCUP ']

yesss
-----------------------------------------------------------------

TARGET CAPTION: tipping bottle over
PRED   CAPTION: POKING  A  STACK  OF  PLASTIC  CUPS  SO  THE  STACK  COLLAPSES 

TARGET  ACTION: TIPPING SOMETHING OVER
CLASSIF ACTION: Poking a stack of [something] so the stack collapses
CAPTION ACTION: POKING A STACK OF SOMETHING SO THE STACK COLLAPSES

TARGET OBJECTS: ['bottle']
PRED   OBJECTS: ['PLASTIC CUPS ']

-----------------------------------------------------------------

TARGET CAPTION: pretending to turn box upside down
PRED   CAPTION: PRETENDING  TO  TURN  A  BOTTLE  UPSIDE  DOWN 

TARGET  ACTION: PRETENDING TO TURN SOMETHING UPSIDE

woohoo
TARGET CAPTION: putting tv remote next to pill bottle
PRED   CAPTION: MOVING  A  CUP  CLOSER  TO  A  BOTTLE 

TARGET  ACTION: PUTTING SOMETHING NEXT TO SOMETHING
CLASSIF ACTION: Moving [something] closer to [something]
CAPTION ACTION: MOVING SOMETHING CLOSER TO SOMETHING

TARGET OBJECTS: ['tv remote', 'pill bottle']
PRED   OBJECTS: ['CUP  ', 'BOTTLE']

-----------------------------------------------------------------

TARGET CAPTION: holding Wallet next to Money
PRED   CAPTION: MOVING  A  BOTTLE  ACROSS  A  SURFACE  UNTIL  IT  FALLS  DOWN 

TARGET  ACTION: HOLDING SOMETHING NEXT TO SOMETHING
CLASSIF ACTION: Moving [something] and [something] closer to each other
CAPTION ACTION: MOVING SOMETHING ACROSS A SURFACE UNTIL IT FALLS DOWN

TARGET OBJECTS: ['Wallet', 'Money']
PRED   OBJECTS: ['BOTTLE ']

-----------------------------------------------------------------

TARGET CAPTION: spinning a lemon that quickly stops spinning
PRED   CAPTION: PUSHING  A  PLASTIC  BOTTLE  SO  IT  SPINS

TARGET CAPTION: a deodorant falling like a rock
PRED   CAPTION: DROPPING  A  BALL  ONTO  THE  FLOOR 

TARGET  ACTION: SOMETHING FALLING LIKE A ROCK
CLASSIF ACTION: [Something] falling like a rock
CAPTION ACTION: DROPPING SOMETHING ONTO SOMETHING

TARGET OBJECTS: ['a deodorant']
PRED   OBJECTS: ['BALL  ', 'FLOOR']

-----------------------------------------------------------------

TARGET CAPTION: rolling paper falling like a feather or paper
PRED   CAPTION: A  BALL  FALLING  LIKE  A  ROCK 

TARGET  ACTION: SOMETHING FALLING LIKE A FEATHER OR PAPER
CLASSIF ACTION: [Something] falling like a rock
CAPTION ACTION: SOMETHING FALLING LIKE A ROCK

TARGET OBJECTS: ['rolling paper']
PRED   OBJECTS: ['BALL ']

-----------------------------------------------------------------

TARGET CAPTION: moving a deodorant up
PRED   CAPTION: MOVING  A  MOBILE  UP 

TARGET  ACTION: MOVING SOMETHING UP
CLASSIF ACTION: Moving [something] up
CAPTION ACTION: MOVING SOMETHING UP

TARGET OBJECTS: ['a deodorant']
PRE

In [30]:
for ca in correct_actions:
    
    print("model got {}/{} of {} actions correct".format(correct_actions[ca], all_actions[ca], ca))

model got 2/5 of POKING SOMETHING SO IT SLIGHTLY MOVES actions correct
model got 1/2 of FAILING TO PUT SOMETHING INTO SOMETHING BECAUSE SOMETHING DOES NOT FIT actions correct
model got 2/7 of HOLDING SOMETHING BEHIND SOMETHING actions correct
model got 2/4 of MOVING SOMETHING AWAY FROM THE CAMERA actions correct
model got 9/16 of DROPPING SOMETHING ONTO SOMETHING actions correct
model got 2/3 of PRETENDING TO POKE SOMETHING actions correct
model got 1/1 of PULLING TWO ENDS OF SOMETHING SO THAT IT SEPARATES INTO TWO PIECES actions correct
model got 2/6 of THROWING SOMETHING ONTO A SURFACE actions correct
model got 2/3 of LIFTING SOMETHING UP COMPLETELY WITHOUT LETTING IT DROP DOWN actions correct
model got 3/3 of LIFTING UP ONE END OF SOMETHING THEN LETTING IT DROP DOWN actions correct
model got 6/10 of PUSHING SOMETHING FROM RIGHT TO LEFT actions correct
model got 11/20 of FOLDING SOMETHING actions correct
model got 3/4 of DROPPING SOMETHING NEXT TO SOMETHING actions correct
model got 

In [35]:
for correct_key in correct_objects.keys():
    denom = 0
    if correct_key in all_objects.keys():
        denom += all_objects[correct_key]
        #print("{}:{}".format(correct_key, all_objects[correct_key]))
    #for j in all_objects.keys():
    #    if correct_key in j or correct_key in j:
    #        print("{}:{}".format(j, all_objects[j]))
    #        denom += all_objects[j]
        
                    
    print (">>model got  {}/{} of '{}'s correct".format(correct_objects[correct_key],denom, correct_key ))
    print("-"*100)
    

>>model got  1/6 of 'CHAIR's correct
----------------------------------------------------------------------------------------------------
>>model got  2/9 of 'CHARGER's correct
----------------------------------------------------------------------------------------------------
>>model got  5/13 of 'SPOON's correct
----------------------------------------------------------------------------------------------------
>>model got  3/7 of 'HAND's correct
----------------------------------------------------------------------------------------------------
>>model got  1/4 of 'USB's correct
----------------------------------------------------------------------------------------------------
>>model got  1/10 of 'MOUSE's correct
----------------------------------------------------------------------------------------------------
>>model got  38/82 of 'PAPER's correct
----------------------------------------------------------------------------------------------------
>>model got  1/5 of 'DRAWER's c

In [52]:
a = sum(correct_objects.values())
b = sum(all_objects.values())

print("{} objects are correctly predicted out of total number of {} objects: {:.2}% ".format(a, b, a/b*100))

185 objects are correctly predicted out of total number of 2341 objects: 7.9% 


In [42]:
a

NameError: name 'a' is not defined

In [38]:
185/2341

0.07902605724049551